In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
audi_df = pd.read_csv('../input/used-car-dataset-ford-and-mercedes/audi.csv')
audi_df.shape

In [ ]:
audi_df.columns

In [ ]:
audi_df.isna().sum()

In [ ]:
audi_df.head()

In [ ]:
audi_df['model'].value_counts()

## **Replacing  model to others which are < 100**

In [ ]:
audi_models = audi_df['model'].value_counts()

others = audi_models[audi_models < 100].keys()

others

In [ ]:
audi_df['model'].replace(others,['Others' for i in range(len(others))], inplace=True)

In [ ]:
plt.figure(figsize=(10, 10))
plt.style.use('fivethirtyeight')
audi_df['model'].value_counts().plot(kind='barh')
plt.title('Audi Models')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.style.use('fivethirtyeight')
audi_df['transmission'].value_counts().plot(kind='barh')
plt.title('Transmission Types')
plt.gca().invert_yaxis()
plt.show()

## Fuel Type 

In [ ]:
audi_df['fuelType'].value_counts()

### Dropping Hybrid fuel type because it is too less 

In [ ]:
audi_df = audi_df[audi_df['fuelType'] != 'Hybrid']

In [ ]:
audi_df.shape

In [ ]:
audi_df.sample(10)

In [ ]:
audi_df[['price', 'mileage', 'tax', 'mpg', 'engineSize']].describe()

In [ ]:
audi_df['year'].unique()

In [ ]:
audi_df['year'].value_counts()

In [ ]:
year_count = audi_df['year'].value_counts()

year_count[year_count<100].sum()

## Replacing years from 1997-2012 to before 2013

In [ ]:
year_keys = year_count[year_count<100].keys()
year_keys

In [ ]:
audi_df['year'].replace(year_keys, ['before 2013' for i in range(len(year_keys))], inplace=True)

In [ ]:
plt.figure(figsize=(10, 10))
plt.style.use('fivethirtyeight')
audi_df['year'].value_counts().plot(kind='barh')
plt.title('Year Frequency')
plt.gca().invert_yaxis()
plt.show()

## Ranking the years 

In [ ]:
years = audi_df['year'].value_counts().keys()

In [ ]:
year_ranks = [i/10 for i in range(1,len(years)+1)]

In [ ]:
audi_df['year'].replace(years, year_ranks, inplace=True)
audi_df['year'].value_counts()

In [ ]:
audi_df.head()

## One Hot encoding 

In [ ]:
cols_to_encode = ['model', 'fuelType','transmission']
df_dummies = pd.get_dummies(audi_df[cols_to_encode],drop_first=True)

## Feature Scaling  

In [ ]:
cols_to_scale = ['mileage', 'tax', 'mpg', 'engineSize']

scale = MinMaxScaler()
scale.fit(audi_df[cols_to_scale])

In [ ]:
scaled = scale.fit_transform(audi_df[cols_to_scale])
scaled

In [ ]:

i = 0
for col in cols_to_scale:
    audi_df[col] = scaled[:,i]
    i += 1

In [ ]:
scaled_cols = audi_df[cols_to_scale]

In [ ]:
df = pd.concat([df_dummies, scaled_cols, audi_df['year'],audi_df['price']], axis=1)

In [ ]:
new_df = df[df['year']!=0.9]

In [ ]:
new_df.shape

## Splitting and Training

In [ ]:
x, y = new_df.drop('price', axis=1), new_df['price']

In [ ]:
x.shape, y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.85,)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
y_train.shape, y_test.shape

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
model.score(x_test, y_test)

In [ ]:
model.score(x_train, y_train)

In [ ]:
min_mil = new_df['mileage'].quantile(0.05)
max_mil = new_df['mileage'].quantile(0.95)

In [ ]:
min_tax = new_df['tax'].quantile(0.05)
max_tax = new_df['tax'].quantile(0.95)

In [ ]:
new_df2 = new_df[(new_df['mileage']>min_mil) & (new_df['mileage']<max_mil)]

In [ ]:
new_df2.shape

In [ ]:
new_df2 = new_df[(new_df['tax']>min_tax) & (new_df['tax']<max_tax)]
new_df2.shape

In [ ]:
new_df2.head()

In [ ]:
x2, y2 = new_df2.drop(['price'], axis=1), new_df2['price']

In [ ]:
x2.shape, y2.shape

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, train_size=0.85,)

In [ ]:
x_train2.shape, x_test2.shape

In [ ]:
y_train2.shape, y_test2.shape

In [ ]:
model2 = LinearRegression()
model2.fit(x_train2, y_train2)

In [ ]:
y_pred_test = model2.predict(x_test2)
y_pred_train = model2.predict(x_train2)

In [ ]:
mean_squared_error(y_test2, y_pred_test)

In [ ]:
mean_squared_error(y_train2, y_pred_train)

In [ ]:
model2.score(x_test2, y_test2)

In [ ]:
model2.score(x_train2, y_train2)

# **Cross val scores**

In [ ]:
reg = cross_val_score(LinearRegression(), x2, y2, cv=10)
reg

In [ ]:
rid = cross_val_score(Ridge(), x2, y2, cv=10)
rid

In [ ]:
las = cross_val_score(Lasso(), x2, y2, cv=10)
las

In [ ]:
reg.mean(), rid.mean(), las.mean()

In [ ]:
svr = cross_val_score(SVR(kernel='linear'), x2, y2, cv=10)
svr

In [ ]:
knr = cross_val_score(KNeighborsRegressor(), x2, y2, cv=10)
knr

In [ ]:
knr = cross_val_score(KNeighborsRegressor(n_neighbors=11,weights='distance',metric='euclidean'), x2, y2, cv=10)
knr

## As we can see  KNeighborsRegressor is giving more score

In [ ]:
kmodel = KNeighborsRegressor(n_neighbors=11,weights='distance',metric='euclidean')
kmodel.fit(x_train2, y_train2)

In [ ]:
kmodel.score(x_test2, y_test2)

In [ ]:
kmodel.score(x_train2, y_train2)

In [ ]:
y_pred_test = model.predict(x_test2)
y_pred_train = model.predict(x_train2)

In [ ]:
test_y = pd.DataFrame({'Y test':y_test2,'Y Pred':y_pred_test})
train_y = pd.DataFrame({'Y test':y_train2,'Y Pred':y_pred_train})


In [ ]:
train_y.head(10)

## **Hyper parameter tuning**

In [ ]:
grid = {
     'n_neighbors':[5, 11, 19, 35, 57],
     'weights':['distance','uniform'],
     'metric':['manhattan','euclidean','minkowski']
}
clf = GridSearchCV(KNeighborsRegressor(),grid, cv=5)

In [ ]:
clf.fit(x_train2, y_train2)

In [ ]:
cv_result = pd.DataFrame(clf.cv_results_)
cv_result

In [ ]:
cv_result[['param_metric', 'param_n_neighbors', 'params', 'mean_test_score', 'mean_test_score']]

In [ ]:
clf.best_estimator_

In [ ]:
kmodel = KNeighborsRegressor(n_neighbors=7, weights='distance',metric='manhattan')
kmodel.fit(x_train2, y_train2)

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, train_size=0.50,)

In [ ]:
kmodel.score(x_test2, y_test2)

In [ ]:
kmodel.score(x_train2, y_train2)

In [ ]:
x_train2.shape

In [ ]:
x_test2.shape

## Omg Score Reached 98 when splitted  data into 50%

In [ ]:
y_pred_test = model.predict(x_test2)
y_pred_train = model.predict(x_train2)
test_y = pd.DataFrame({'Y test':y_test2,'Y Pred':y_pred_test})
train_y = pd.DataFrame({'Y test':y_train2,'Y Pred':y_pred_train})

In [ ]:
test_y.sample(10)

In [ ]:
test_y.corr()

In [ ]:
train_y.sample(10)

In [ ]:
train_y.corr()